# Episodio 12: definizione e istanziamento di typeclass, parte 1
Gli esempi qui sotto possono essere eseguiti copiandoli nell'interprete di Haskell, il `ghci`. Se vuoi eseguire direttamente questo notebook, le istruzioni su come farlo sono disponibili a [questo](https://github.com/gibiansky/IHaskell) link.

Nella scorsa lezione, abbiamo istanziato una typeclass preesistente, `Show`. Abbiamo anche visto la sintassi per definire nuove typeclass. In questa lezione vedremo un esempio pratico di definizione ed istanziamento di una typeclass non standard, ovvero di una typeclass che non è definita sotto altro nome in nessuna delle librerie di Haskell di uso comune.
L'esempio è fortemente ispirato alla [typeclass `YesNo` di _Learn You A Haskell, For Great Good!_](http://learnyouahaskell.com/making-our-own-types-and-typeclasses#a-yes-no-typeclass), che prende spunto a sua volta da alcune cose che si possono fare in linguaggi debolmente tipizzati come il JavaScript e il Python, ma che il fortissimo type system di Haskell non permette.

## La caratteristica del Python che cercheremo di imitare
In Python, è possibile scrivere condizionali come il seguente:

```python
if stringa:
    print(stringa)
else:
    print("mi dispiace, la stringa è vuota!")
```

(o, come ho scritto in una linea nel video, `print(stringa if stringa else "mi dispiace, la stringa è vuota")`).

Il comportamento di questo codice varia a seconda che la stringa sia vuota (`""`) o "piena" (per es. `"ciao"`). Nel primo il risultato sarà un bel `mi dispiace, la stringa è vuota!`, mentre se la stringa non è vuota ne verrà visualizzato il contenuto.
Questo significa che, in un `if`, le stringhe vengono trattate un po' come booleani: se vuote, vengono considerate "false", altrimenti "vere".

Questo ragionamento si applica non solo alle stringhe, ma a quasi tutti i tipi: numeri, liste, dizionari...

## Una typeclass "booleaneggiante" in Haskell
Con la premessa che il rendere confrontabili booleani con tipi arbitrari é a mio avviso un'idea __nefasta__, cercheremo di replicare questa feature in Haskell tramite una typeclass, che chiameremo `Verificabile`. Questa typeclass avrà due funzioni, `verificato` e `falsificato`, definite l'una in termini dell'altra in modo che, all'istanziamento, sia sufficiente specificare il comportamento di solo una delle due per poter ottenere automaticamente anche l'altra:

In [20]:
class Verificabile a where
    verificato :: a -> Bool
    verificato a = not (falsificato a)

    falsificato :: a -> Bool
    falsificato a = not (verificato a) 

La variabile di tipo `a`, senza alcun constraint, significa che qualunque tipo può essere istanza di `Verificabile`.
Il "default" di `verificato` è definito come l'opposto di `falsificato` e viceversa.

## Istanziamento di `Verificabile`

### Bool
Per prima cosa, ovviamente, rendiamo `Bool` istanza di `Verificabile`:

In [21]:
instance Verificabile Bool where
    verificato True = True
    verificato False = False

Facile, no? Come anticipato, non é necessario definire anche `falsificato` per via della definizione che abbiamo scritto all'interno della typeclass stessa. Difatti possiamo scrivere:

In [22]:
falsificato True

False

In [23]:
falsificato False

True

Passiamo ora ad istanziare la nostra nuova typeclass per altri tipi di dato.

### Int

In [24]:
instance Verificabile Int where
    verificato 0 = False
    verificato _ = True

In [25]:
n = 0 :: Int
n' = 15 :: Int

In [26]:
verificato n

False

In [27]:
verificato n'

True

### Liste

In [28]:
instance Verificabile [a] where
    verificato [] = False
    verificato _ = True

In [29]:
verificato []

False

In [30]:
verificato [1,2,3]

True

Ovviamente, questo fa sì che `verificato` e `falsificato` funzionino anche per le stringhe, che sono liste di caratteri. Di conseguenza, possiamo replicare l'esempio che abbiamo visto all'inizio in Python:

In [31]:
esempioPython stringa = 
    if verificato stringa -- particamente basta aggiungere un "verificato" qui
        then putStrLn stringa
        else putStrLn "mi dispiace, la stringa è vuota!"

(NB: equivalentemente ma in una sola riga, in GHCI ho scritto `putStrLn (if verificato stringa then stringa else "mi dispiace, la stringa è vuota")`)

In [32]:
esempioPython "ciao"

ciao

In [33]:
esempioPython ""

mi dispiace, la stringa è vuota!

### Maybe

In [34]:
instance Verificabile (Maybe a) where
    verificato Nothing = False
    verificato _ = True

In [35]:
verificato Nothing

False

In [36]:
verificato (Just "ciao")

True

In [37]:
falsificato Nothing

True

In [38]:
falsificato (Just "ciao")

False